In [1]:
using RxEnvironments
using Distributions

# Empty agent, could contain states as well
struct ThermostatAgent end

mutable struct BayesianThermostat{T}
    temperature::T
    min_temp::T
    max_temp::T
end

# Helper functions
temperature(env::BayesianThermostat) = env.temperature
min_temp(env::BayesianThermostat) = env.min_temp
max_temp(env::BayesianThermostat) = env.max_temp
noise(env::BayesianThermostat) = Normal(0.0, 0.1)
set_temperature!(env::BayesianThermostat, temp::Real) = env.temperature = temp
function add_temperature!(env::BayesianThermostat, diff::Real)
    env.temperature += diff
    if temperature(env) < min_temp(env)
        set_temperature!(env, min_temp(env))
    elseif temperature(env) > max_temp(env)
        set_temperature!(env, max_temp(env))
    end
end

add_temperature! (generic function with 1 method)

In [2]:
# When the environment receives an action from the agent, we add the value of the action to the environment temperature.
RxEnvironments.receive!(
    recipient::BayesianThermostat, 
    emitter::ThermostatAgent, 
    action::Real
) = add_temperature!(recipient, action)

# The environment sends a noisy observation of the temperature to the agent.
RxEnvironments.what_to_send(
    recipient::ThermostatAgent, 
    emitter::BayesianThermostat
) = temperature(emitter) + rand(noise(emitter))

# The environment cools down over time.
RxEnvironments.update!(
    env::BayesianThermostat, 
    elapsed_time
)= add_temperature!(env, -0.1 * elapsed_time)

In [3]:
# environment = RxEnvironment(
#     BayesianThermostat(0.0, -10.0, 10.0);
#     emit_every_ms = 900
# )

environment = create_entity(BayesianThermostat(0.0, -10.0, 10.0); is_discrete=false, is_active=true)

Continuous RxEntity{BayesianThermostat{Float64}}

In [4]:
agent = add!(environment, ThermostatAgent())

Continuous RxEntity{ThermostatAgent}

In [5]:
typeof(agent)

RxEnvironments.RxEntity{ThermostatAgent, RxEnvironments.ContinuousEntity, RxEnvironments.PassiveEntity, Any}

In [6]:
typeof(environment)

RxEnvironments.RxEntity{BayesianThermostat{Float64}, RxEnvironments.ContinuousEntity, RxEnvironments.ActiveEntity, Any}

In [7]:
# Subscribe a logger actor to the observations of the agent
RxEnvironments.subscribe_to_observations!(agent, RxEnvironments.logger())

# Conduct 10 actions:
for i in 1:10
    action = rand()
    RxEnvironments.send!(environment, agent, action)
    sleep(1)
end

[LogActor] Data: -1.0528827140390709
[LogActor] Data: 0.04970466045173649
[LogActor] Data: 0.315747687471174
[LogActor] Data: 1.1156524094142382
[LogActor] Data: 1.442781287385217
[LogActor] Data: 1.342729673009397
[LogActor] Data: 1.2490263921010962
[LogActor] Data: 1.7959385166664825
[LogActor] Data: 2.0349544318425377
[LogActor] Data: 2.247610860675016
